# Рекомендательные системы

## Курсовой проект

Результат: precision@5 от 0.2520 до 0.2546 в разные запуски (фактор рандомности, видимо)

**Примечания**

- в домашних работах частично переписывал код методичек для лучшего усвоения материала, в курсовом старался вносить минимальные изменения в предложенный код
- тем не менее, убрал иллюстративные и отладочные фрагменты, немного перекомпоновал блоки
- в целом данное начальное знакомство с направлением рекомендательных систем оставило впечатление определенного шаманства - много методов, все логичные, некоторые - с хорошим математическим обоснованием, но предсказать, что именно и как сработает без опыта практически невозможно 

Внесенные изменения:

- N_PREDICT = 9
- Заменил prefilter_items на собственную версию, число items получилось в районе 4000, а не 5000
- Чуть подкрутил BM25 в MainRecommender
- Убрал user_id и item_id из модели второго уровня (кажется, это немного снизило результат, но ID не принято отдавать в бустинг)
- Раскомментировал все предложенные фичи (прибавка есть, хотя и весьма небольшая)
- Добавил значение, которое выдает ALS, в качестве фичи
- Выставил learning_rate=0.07
- Убрал раздел с result_eval_ranker для увеличения скорости
- Переделал фунцию rerank так, чтобы в reranked_own_rec не было пустых рекомендаций (кстати, это снизило precision@5)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from metrics import precision_at_k, recall_at_k

# С правками для курсового:
from student_utils import prefilter_items
from student_recommenders import MainRecommender

In [2]:
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [3]:
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [4]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'

N_PREDICT = 9

TOPK_RECALL = 50
TOPK_PRECISION = 5

VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

## Read data

In [5]:
data = pd.read_csv('../hw2/retail_train.csv.zip')
item_features = pd.read_csv('../hw2/product.csv')
user_features = pd.read_csv('../hw5/hh_demographic.csv')
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]
item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)
df_test = pd.read_csv('retail_test1.csv')

# Split dataset for train, eval, test

In [6]:
# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]

# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [7]:
# сделаем объединенный сет данных для первого уровня (матчинга)
df_join_train_matcher = pd.concat([data_train_matcher, data_val_matcher])

# Prefilter items

In [8]:
n_items_before = data_train_matcher['item_id'].nunique()

#data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=5000)
data_train_matcher = prefilter_items(data_train_matcher, prevalence_range = (0.09, 0.61), price_range = (0.5, 50.0))

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 3921


# Make cold-start to warm-start

In [9]:
# ищем общих пользователей
common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]
df_test = df_test[df_test.user_id.isin(common_users)]

# Init/train recommender

In [10]:
recommender = MainRecommender(data_train_matcher)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/3921 [00:00<?, ?it/s]

# Eval recall of matching

In [11]:
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [12]:
%%time
result_eval_matcher['own_rec'] = result_eval_matcher[USER_COL].apply(
    lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

Wall time: 34.7 s


### Recall@50 of matching

In [13]:
next(calc_recall(result_eval_matcher, TOPK_RECALL))

('own_rec', 0.0486123934714024)

### Precision@5 of matching

In [14]:
next(calc_precision(result_eval_matcher, TOPK_PRECISION))

('own_rec', 0.3211333023687878)

# Ranking part

In [15]:
# взяли пользователей из трейна для ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [16]:
# собираем кандитатов с первого этапа (matcher)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(
    lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

In [17]:
# разворачиваем товары
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

### Создаем трейн сет для ранжирования с учетом кандидатов с этапа 1 

In [18]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки 

df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

df_ranker_train['target'].fillna(0, inplace= True)

## Подготавливаем фичи для обучения модели

In [19]:
df_ranker_train = df_ranker_train.merge(item_features, on=ITEM_COL, how='left')
df_ranker_train = df_ranker_train.merge(user_features, on=USER_COL, how='left')

In [20]:
cat_feats = df_ranker_train.columns[3:].tolist()

In [21]:
# ALS feature
fast_recs = recommender.model.user_factors @ recommender.model.item_factors.T
df_ranker_train['als_score'] = df_ranker_train.apply(
    lambda x: fast_recs[recommender.userid_to_id[x.user_id], 
                        recommender.itemid_to_id[x.item_id]], 
    axis=1)

### Поведенческие фичи


In [22]:
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('sales_value').sum().rename('total_item_sales_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('total_quantity_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('sales_value').sum().rename('total_user_sales_value'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=USER_COL)


df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_baskter')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)


df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)


In [23]:
X_train = df_ranker_train.drop(['target', USER_COL, ITEM_COL], axis=1)
y_train = df_ranker_train['target']
X_train[cat_feats] = X_train[cat_feats].astype('category')

## Обучение модели ранжирования

In [24]:
lgb = LGBMClassifier(objective='binary',
                     max_depth=10,
                     n_estimators=500,
                     learning_rate=0.07)

lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)

In [25]:
df_ranker_predict = (
    df_ranker_train
    .filter([USER_COL, ITEM_COL])
    .assign(proba=train_preds[:,1])
)

# Оценка на тесте для выполнения курсового проекта

In [26]:
result_test = df_test.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_test.columns=[USER_COL, ACTUAL_COL]

Берем топ-k предсказаний, ранжированных по вероятности, для каждого юзера

In [27]:
def reranked(recommendations, user_item_proba, prefix = 'reranked_'):
    rec_col = recommendations.columns[-1]
    proba_col = user_item_proba.columns[-1]
    _uip = user_item_proba.filter([USER_COL, ITEM_COL, proba_col])

    return (
        recommendations
        .rename(columns={rec_col:ITEM_COL})
        .explode(ITEM_COL)
        .assign(orig_rank = lambda x: x.groupby(level=-1).cumcount())
        .merge(_uip, on=[USER_COL, ITEM_COL], how='left')
        .assign(sort_factor = lambda x: np.where(x[proba_col].isna(), x.orig_rank, -x[proba_col]))
        .sort_values([USER_COL, 'sort_factor'], ascending=True)
        .groupby(USER_COL)
        .agg({ITEM_COL: list})
        .rename(columns={ITEM_COL:f"{prefix}{rec_col}"})
        .reset_index()
    )

In [28]:
%%time
result_test['own_rec'] = result_test[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
#result_test['reranked_own_rec'] = result_test[USER_COL].apply(lambda user_id: rerank(user_id))
result_test = result_test.merge(reranked(result_test, df_ranker_predict), on=USER_COL)

Wall time: 34 s


In [29]:
print(*sorted(calc_precision(result_test, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.2546709129511677)
('own_rec', 0.23365180467091298)
